In [ ]:
import os

# 기본 경로 설정
root_path = '/data/ephemeral/home'
dir_name = 'dataset_final'

# 이미지와 레이블의 서브 디렉토리 경로 설정
image_dirs = ['images/train', 'images/valid', 'images/test']
label_dirs = ['labels/train', 'labels/valid']

# 모든 필요한 디렉토리 생성
for dir in image_dirs + label_dirs:
    os.makedirs(os.path.join(root_path, dir_name, dir), exist_ok=True)

In [ ]:

import shutil
import os
import glob

# KFold 번호 설정
kfold_number = 0

# 기존 및 새 디렉토리 경로 설정
src_train_images = os.path.join(root_path, 'dataset/train')
src_test_images = os.path.join(root_path, 'dataset/test')
dest_train_images = os.path.join(root_path, dir_name, 'images/train')
dest_test_images = os.path.join(root_path, dir_name, 'images/test')
src_train_json = os.path.join(root_path, f'dataset/train_kfold_{kfold_number}.json')
src_val_json = os.path.join(root_path, f'dataset/val_kfold_{kfold_number}.json')
src_test_json = os.path.join(root_path, 'dataset/test.json')
dest_json = os.path.join(root_path, dir_name)

# train 이미지 복사
for file in glob.glob(f'{src_train_images}/*.jpg'):
    shutil.copy(file, dest_train_images)

# test 이미지 복사
for file in glob.glob(f'{src_test_images}/*.jpg'):
    shutil.copy(file, dest_test_images)

# JSON 파일 복사
shutil.copy(src_train_json, dest_json)
shutil.copy(src_val_json, dest_json)
shutil.copy(src_test_json, dest_json)

In [ ]:

train_json_path = os.path.join(root_path, dir_name, f'train_kfold_{kfold_number}.json')
val_json_path = os.path.join(root_path, dir_name, f'val_kfold_{kfold_number}.json')
label_path = os.path.join(root_path, dir_name, 'labels')

!python coco2yolo.py -j {train_json_path} -o {label_path} -f train
!python coco2yolo.py -j {val_json_path} -o {label_path} -f val

In [ ]:
# 검증 세트 레이블이 저장된 디렉토리
valid_label_dir = os.path.join(root_path, dir_name, 'labels/valid')

# 이미지 디렉토리
train_image_dir = os.path.join(root_path, dir_name, 'images/train')
valid_image_dir = os.path.join(root_path, dir_name, 'images/valid')

# valid 레이블에 해당하는 이미지 파일 찾기
valid_labels = glob.glob(f'{valid_label_dir}/*.txt')

for label_path in valid_labels:
    image_path = label_path.replace('labels', 'images').replace('valid', 'train').replace('.txt', '.jpg')

    if os.path.exists(image_path):
        shutil.move(image_path, valid_image_dir)
    else:
        print('No image file exist ! : ', image_path)